# Librerias

In [2]:
import numpy as np
from torchvision.datasets import MNIST
from descenso_gradiente import descenso_gradiente
from time import perf_counter_ns
import pandas as pd

# Obteniendo el dataset

In [3]:
df_train: list = list(zip(*MNIST(root="",
                                 train=True,
                                 download=True,
                                 transform=lambda t: np.asarray(t, dtype=np.float32))))

df_test: list = list(zip(*MNIST(root="",
                                train=False,
                                download=True,
                                transform=lambda t: np.asarray(t, dtype=np.float32))))


In [4]:
# Separamos las imagenes (x) de los labels (y)
df_trainX: np.ndarray = np.asarray(df_train[0])
df_trainY: np.ndarray = np.asarray(df_train[1])

df_testX: np.ndarray = np.asarray(df_test[0])
df_testY: np.ndarray = np.asarray(df_test[1])

In [50]:
list(map(type,[1,2]))

[int, int]

# Función de Entrenamiento

In [5]:
function_train: dict[str,callable] = {"step": descenso_gradiente}

In [6]:
def funcion_objetivo():
    pass

# Main

In [20]:
def initial_clean_df(list_comb_num: list[tuple[int,int]]) -> None:
    name_df_main = "DF_ALGORITHMS.csv"
    name_df_comb = "DF_DATA_COMBINATION.csv"
    name_df_iter = "DF_ITERATIONS.csv"
    
    global df_main
    global df_comb
    global df_iter

    # Dataframe que contendrá los datos de cada prueba de combinación
    columns_df_main: list = ["algorithm","time","num_iter","value_grad","data_index"]

    df_main = pd.DataFrame(columns=columns_df_main)
    df_main.to_csv(name_df_main)

    # Dataframe que contendrá los datos de cada iteración
    columns_df_iter: list = ["main_index","time_iter","value_grad_iter","const_iter"]

    df_main = pd.DataFrame(columns=columns_df_iter)
    df_main.to_csv(name_df_iter)

    
    df_comb = pd.DataFrame(list_comb_num, columns=["number_1","number_2"], dtype=np.int8)
    df_comb.to_csv(name_df_comb)

In [22]:
# Todas las comparaciones entre las clases
comb_num = []
for x in range(9):
    for y in range(9):
        if x!=y:
            comb_num.append((x,y))   
            
initial_clean_df(comb_num)

SyntaxError: invalid decimal literal (3124438653.py, line 1)

In [37]:
1.(_type_) is int

<>:1: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
C:\Users\Sebastian\AppData\Local\Temp\ipykernel_14952\314665044.py:1: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?
  1.(_type_) is int


NameError: name '_type_' is not defined

In [63]:
a = [(1,2),(2,3), (11,22),(12,33),(12, 1)]
s,b = zip(*a)
set(s)

{1, 2, 11, 12}

In [8]:
# Escogemos los valores con que queremos entrenar nuestra función
n0: int = 0
n1: int = 1

# Filtramos el dataset de entrenamiento y de testeo para obtener los datos que tengan las etiquetas n0 y n1
x_train: np.ndarray = df_trainX[(df_trainY == n0) | (df_trainY == n1)]
y_train: np.ndarray = df_trainY[(df_trainY == n0) | (df_trainY == n1)]
x_test: np.ndarray = df_testX[(df_testY == n0) | (df_testY == n1)]
y_test: np.ndarray = df_testY[(df_testY == n0) | (df_testY == n1)]

# Cambiamos las etiquetas del conjunto de las clases (y)
# no → 1 y n1 → -1
y_train = np.where(y_train == n0, 1, -1)
y_test = np.where(y_test == n0, 1, -1)

# Aplicamos algún tipo de perturbación a los datos de entrenamiento (x)
fun_per: callable = lambda t: t + 1 # Decidimos la función de perturbation
v_fun_per = np.vectorize(fun_per) # Vectorizamos la función para ejecutarla en todo el array

x_train = v_fun_per(x_train)
x_test = v_fun_per(x_test)

# Ya listo el dataset y las clases, se procede a encontrar los hiperparámetros de que minimizan el error
# entre una función y las clases. Para esto podemos usar descenso del gradiente u otro método.

#Tomamos el tiempo de inicio
t_ini = perf_counter_ns()
W_opt, num_iter, value_grad = descenso_gradiente()
t_fin = perf_counter_ns() - t_ini










TypeError: descenso_gradiente() missing 3 required positional arguments: 'fun', 'xk', and 'grad_fun'

In [30]:
pd.read_csv("DF_DATA_COMBINATION.csv", index_col="Unnamed: 0")

,Unnamed: 0,number_1,number_2
0,0,0,1
1,1,0,2
2,2,0,3
3,3,0,4
4,4,0,5
...,...,...,...
67,67,8,3
68,68,8,4
69,69,8,5
70,70,8,6


In [ ]:
x_train